### Prerequisites
* Install the FastChat package by following the instructions in the parent directory's [README.md](../README.md) file or follow the [tutorial.ipynb](tutorial.ipynb) notebook's Installation section.
* Get vicuna weights for a chatbot by following the instructions in the [vicuna_weights.ipynb](vicuna_weights.ipynb) notebook.


## Inference with Command Line Interface
The FastChat CLI provides a command-line interface for inference. You can specify different options to configure the inference process.

### Single GPU
The command below requires around 28GB of GPU memory for Vicuna-13B and 14GB of GPU memory for Vicuna-7B.
See the "No Enough Memory" section below if you do not have enough memory.

In [ ]:
vicuna_weight_path= # /path/to/vicuna/weights
!python3 -m fastchat.serve.cli --model-path {vicuna_weight_path} 

### Multiple GPUs
If you do not have enough GPU memory, you can use model parallelism to aggregate memory from multiple GPUs on the same machine.


In [ ]:
vicuna_weight_path= # /path/to/vicuna/weights
!python3 -m fastchat.serve.cli  --num-gpus 2 --model-path {vicuna_weight_path}

### No Enough Memory or Other Platforms
If you do not have enough memory, you can enable 8-bit compression by adding `--load-8bit` to commands above.
This can reduce memory usage by around half with slightly degraded model quality.
It is compatible with the CPU, GPU, and Metal backend.
Vicuna-13B with 8-bit compression can run on a single NVIDIA 3090/4080/V100(16GB) GPU.

In [ ]:
vicuna_weight_path = # /path/to/vicuna/weights
!python3 -m fastchat.serve.cli  --load-8bit --model-path {vicuna_weight_path} 

## Serving with Web GUI

To serve using the web UI, you need three main components: web servers that interface with users, model workers that host one or more models, and a controller to coordinate the webserver and model workers. Here are the commands to follow in your terminal:


### Launch the controller
The controller is responsible for coordinating the webserver and model workers. It needs to be launched first.

In [ ]:
import subprocess # we need this to run the controller in a separate process in the jupyter notebook
subprocess.Popen(["python3", "-m", "fastchat.serve.controller"])

### Launch the model worker

In [ ]:

vicuna_weight_path = # /path/to/vicuna/weights
device = "cuda" # or "cpu" / "mps"
subprocess.Popen(["python3", "-m", "fastchat.serve.model_worker","--device",device, "--model-path", vicuna_weight_path])


If you do not have enough memory, you can enable 8-bit compression by adding `--load-8bit` to commands above.

Wait until the process finishes loading the model and you see **"Uvicorn running on ..."**. You can launch multiple model workers to serve multiple models concurrently. The model worker will connect to the controller automatically.

To ensure that your model worker is connected to your controller properly, send a test message using the following command:

In [ ]:
!python3 -m fastchat.serve.test_message --model-name vicuna-7b

### Launch the Gradio web server
This is the user interface that users will interact with.

In [ ]:
subprocess.Popen(["python3", "-m", "fastchat.serve.gradio_web_server"])

### Kill the processes
To kill the processes, you can use the following commands:

In [ ]:

!kill -9 $(lsof -t -i:21001) # kill the controller
!kill -9 $(lsof -t -i:21002) # kill the worker
!kill -9 $(lsof -t -i:7860) # kill the gradio server
